In [8]:
import os
from datetime import datetime
from dotenv import load_dotenv
from typing import Annotated, TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate

# Hybrid Switcher Imports
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from langchain_core.tools import tool

load_dotenv()

# --- 1. SETUP & TOOLS ---
def get_llm():
    llm_type = os.getenv("LLM_TYPE", "ollama")
    if llm_type == "groq":
        # Groq is fast, perfect for routing
        return ChatGroq(model="llama-3.1-8b-instant", temperature=0)
    else:
        return ChatOllama(model="llama3.1", temperature=0)

@tool
def get_current_time():
    """Returns the current time."""
    return datetime.now().strftime("%H:%M:%S")

@tool
def multiply_numbers(a: float, b: float):
    """Multiplies two numbers."""
    return a * b

@tool
def get_system_info():
    """Returns OS info."""
    import platform
    return platform.system()

tools = [get_current_time, multiply_numbers, get_system_info]
llm = get_llm()

# --- 2. DEFINE THE STATE ---
class AgentState(TypedDict):
    # 'add_messages' ensures history is preserved
    messages: Annotated[list, add_messages]

# --- 3. DEFINE THE NODES ---

# NODE A: THE CHATBOT (No Tools Allowed!)
# This node handles greetings, names, and chit-chat.
def chat_node(state: AgentState):
    return {"messages": [llm.invoke(state["messages"])]}

# NODE B: THE SPECIALIST (Has Tools)
# This node handles math and system checks.
def specialist_node(state: AgentState):
    llm_with_tools = llm.bind_tools(tools)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

def secret_node(state: AgentState):
    return {"messages": [AIMessage(content="I am not allowed to tell you the secret, but I like your style.")]}

# --- 4. THE ROUTING LOGIC ---
# This function runs BEFORE any node. It decides which path to take.
def router_logic(state: AgentState):
    last_message = state["messages"][-1]
    text = last_message.content.lower()
    
    # 1. Fast Keyword Check (Saves money/time)
    keywords = ["calculate", "multiply", "time", "os", "system", "math", "check"]
    if any(k in text for k in keywords):
        return "specialist"

    if any(word in text for word in ["secret"]):
        return "secret"

    # 2. Smart Check (Ask the LLM)
    # If keywords fail, we ask the brain "Is this a tool request?"
    router_prompt = ChatPromptTemplate.from_messages([
        ("system", "Is this query regarding math, time, or system info? Answer purely YES or NO."),
        ("human", "{input}")
    ])
    chain = router_prompt | llm
    response = chain.invoke({"input": text})
    
    if "YES" in response.content.upper():
        return "specialist"
    else:
        return "chat_agent"

# --- 5. BUILD THE GRAPH ---
workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("chat_agent", chat_node)
workflow.add_node("specialist", specialist_node)
workflow.add_node("tools", ToolNode(tools))
workflow.add_node("secret", secret_node)

# Add Conditional Edge from START
# "When we start, run 'router_logic', then go to either 'chat_agent' or 'specialist'"
workflow.add_conditional_edges(
    START,
    router_logic,
    {
        "chat_agent": "chat_agent",
        "specialist": "specialist",
        "secret": "secret"
    }
)

# Chat Agent always ends the turn
workflow.add_edge("chat_agent", END)

# Specialist Loop logic
def specialist_routing(state):
    last_message = state["messages"][-1]
    # If the specialist asked for a tool, go to 'tools' node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, end
    return END

workflow.add_conditional_edges("specialist", specialist_routing, ["tools", END])
workflow.add_edge("tools", "specialist") # Tools always go back to specialist

# --- 6. COMPILE & RUN ---
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

def run_agent(query, thread_id="default"):
    print(f"\nUser ({thread_id}): {query}")
    config = {"configurable": {"thread_id": thread_id}}
    
    # FIX: Added defensive check 'if value is not None'
    for event in app.stream({"messages": [HumanMessage(content=query)]}, config=config):
        for key, value in event.items():
            if value is not None and "messages" in value:
                last_msg = value["messages"][-1]
                # Only print if it's an AI message with actual text content
                if isinstance(last_msg, AIMessage) and last_msg.content:
                     print(f"  🤖 Node '{key}' says: {last_msg.content}")

# --- TEST SUITE ---

# 1. Identity (Should go to Chat Node -> NO tools -> No JSON panic)
run_agent("My name is Captain Jack.", thread_id="thread_1")

# 2. Recall (Should go to Chat Node)
run_agent("What is my name?", thread_id="thread_1")

# 3. Work (Should go to Specialist Node)
run_agent("What is 100 * 50?", thread_id="thread_1")

# 4. Complex Work
run_agent("can you tell me the secret.", thread_id="thread_1")


User (thread_1): My name is Captain Jack.
  🤖 Node 'chat_agent' says: Captain Jack, eh? Sounds like a swashbuckling adventurer to me! Are you ready to set sail for new horizons and face whatever dangers lie ahead? Or perhaps you're just looking for a friendly chat with a trusty AI companion? Either way, I'm here to listen and respond as your loyal first mate. What's on your mind, Captain Jack?

User (thread_1): What is my name?
  🤖 Node 'chat_agent' says: Your name is Captain Jack!

User (thread_1): What is 100 * 50?
  🤖 Node 'specialist' says: The result of multiplying 100 by 50 is 5000.

User (thread_1): can you tell me the secret.
  🤖 Node 'secret' says: I am not allowed to tell you the secret, but I like your style.
